# Preparing the Data

In [1]:
# Set your working directory
from os import getcwd
import pandas as pd
import numpy as np
import pickle as pk
from nltk import stem
import json
import matplotlib.pyplot as plt

fpath = getcwd()
print (fpath)

/Users/pierlim/PycharmProjects/sent_mining_CA


In [3]:
datapath = fpath + '/data/'
yjson = json.load(open(datapath + "yelp_2.json", "rb"))  # there are 35 reviews in this json
reviews = {}
i=0
for key0, val0 in yjson.items():  # keys are 0, 20  etc..
    for val in val0["review"]:
        reviews[i]= {"rating" : val["reviewRating"]["ratingValue"],
                     "review" : val["description"]}
        i += 1
print ("Total no of reviews " + str(len(reviews)))

Total no of reviews 35


## Reading Scraped Yelp Reviews 

In [82]:
# Read train data (6 restaurants out of 10 restaurants for each category) and collate 

import os
total_reviews = {}
directory = '/Users/pierlim/PycharmProjects/sent_mining_CA/data/chinese_100_reviews_each/'

for filename in os.listdir(directory):
    
    if filename.endswith(".json"):
        print(filename)
        yjson = json.load(open(directory + filename, "rb")) 
        reviews = {}
        i=0
        for key0, val0 in yjson.items():  # keys are 0, 20  etc..
            for val in val0["review"]:
                reviews[i]= {"rating" : val["reviewRating"]["ratingValue"],
                             "review" : val["description"]}
                i += 1
        if bool(total_reviews)==False:
            total_reviews = dict(reviews)
        else:
            # need to re-enumerate the numbers here, else it will overwrite the total_reviews by key!
            reviews_to_update = {}
            cnt = len(total_reviews)
            for k,v in reviews.items():
                reviews_to_update[str(cnt)] = v
                cnt = cnt + 1
            total_reviews.update(reviews_to_update)
            #print(total_reviews)
            


yelp_chinese_1_Lanzhou Noodle Bar.json
Total no of reviews 100 for yelp_chinese_1_Lanzhou Noodle Bar.json
yelp_chinese_3_Wong Kei Restaurant.json
Total no of reviews 100 for yelp_chinese_3_Wong Kei Restaurant.json
updating...
yelp_chinese_8_Golden Dragon.json
Total no of reviews 100 for yelp_chinese_8_Golden Dragon.json
updating...
yelp_chinese_2_Hakkasan.json
Total no of reviews 100 for yelp_chinese_2_Hakkasan.json
updating...
yelp_chinese_0_Yauatcha.json
Total no of reviews 100 for yelp_chinese_0_Yauatcha.json
updating...
yelp_chinese_9_Four Seasons.json
Total no of reviews 100 for yelp_chinese_9_Four Seasons.json
updating...
yelp_chinese_5_Ping Pong.json
Total no of reviews 100 for yelp_chinese_5_Ping Pong.json
updating...
yelp_chinese_7_Ping Pong Soho.json
Total no of reviews 100 for yelp_chinese_7_Ping Pong Soho.json
updating...
yelp_chinese_6_Joy King Lau.json
Total no of reviews 120 for yelp_chinese_6_Joy King Lau.json
updating...
yelp_chinese_4_Dumplings’ Legend.json
Total no o

In [83]:
#print(total_reviews)
#print ("Total no of reviews " + str(len(total_reviews)))
import json
with open('total_reviews.json', 'w') as outfile:
    json.dump(total_reviews, outfile)

In [54]:

yjson = json.load(open(datapath + "yelp_french_0_Sketch.json", "rb")) 
chin_reviews = {}
i=0
for key0, val0 in yjson.items():  # keys are 0, 20  etc..
    for val in val0["review"]:
        chin_reviews[i]= {"rating" : val["reviewRating"]["ratingValue"],
                     "review" : val["description"]}
        i += 1
print ("Total no of reviews " + str(len(chin_reviews)))

Total no of reviews 580


In [55]:
chin_reviews[4]

{'rating': 1,
 'review': 'Worst, worst, worst!\nI definitely do not have anything great to say about sketch.\nMy husband and I came here back two Saturday nights ago; we were both super hyped about dining here. Unfortunately the evening was unforgettable for all the wrong reasons.\nWe arrived for our reservation at 10:00 p.m. and were promptly sat at our table. The server handed us our menus and said that she would be back.\nWe wanted to start our meal with a bottle of wine which we had to wait for the sommelier to come over to actually order(mind you we already were set with what we wanted) \nFinally it came time to order our meals. My husband tried to order the steak and was told they were out for the evening; he proceeded to ordered the lamb, and yet again we were told they were out of it! Why was this not told to us prior to placing our order?! We definitely would have left immediately. He ended up having to order the bone marrow(not great and lacked flavor)\nI ordered the tortelli

## Create the class labels -1 (neg), 1 (pos)
Note that we should not use lecturer-supplied train_neg.csv and train_pos.csv as the criteria for -ve and +ve may be different. 

We need to build our own training set based on what we scraped as well as the yelp_2.json

Use the ratings 1-3 as negative and 4-5 as positive in the Yelp reviews.

In [26]:
def mapPosNegR(score):
    rate = 1
    if score <= 3: rate = -1   # the result is dependent on the map fn
    return rate

actuals = []
for key, val in reviews.items():  # going through each review
    actuals.append(val['rating'])
    
actuals = list(map(mapPosNegR, actuals))  # actuals in terms of +1 or -1

In [27]:
df_reviews = pd.DataFrame(reviews).transpose()

In [28]:
df_reviews = pd.concat([df_reviews, pd.DataFrame(actuals)], axis=1)
#df_reviews.columns = {'rating', 'review', 'sentiment'}

In [30]:
df_reviews.rename(columns={0:'sentiment'}, 
                 inplace=True)
df_reviews.head()

,rating,review,sentiment
0,5,The service at Sushi Bar was great with an ext...,1
1,2,OMG Yelp let me down big time. I don't know h...,-1
2,4,Heard that this was the best sushi joint in Si...,1
3,3,Ebi fry salmon Aburizal ($17.90++) - 3 stars\n...,-1
4,5,Yes I also get the raving reviews now.\nWe dec...,1


Most reviews here are positive according to our rating criteria: it is an imbalanced set. 

In [38]:
df_reviews.groupby('sentiment').sum()

,rating,review
sentiment,,
-1,11,OMG Yelp let me down big time. I don't know h...
1,141,The service at Sushi Bar was great with an ext...


In [43]:
df_reviews
df_reviews.to_csv('./data/df_reviews.csv', index=False) # TODO this needs to be updated with scraped data 
